In [ ]:
# Start with loading all necessary libraries
import numpy as np
import pandas as pd
from os import path
import re

import seaborn as sns
sns.set()

import matplotlib.pyplot as plt
%matplotlib inline



# Let's read some data

In [ ]:
df = pd.read_excel("data/Desk Preferences - anonymised.xlsx", index_col=0)
df.sample(5)

# Cleaning Teams

We purposefully made the teams free-text to see what people would write. For example, there are some pure gems such as:

In [ ]:
df.iloc[41]["Home Team"]

Anyway - it means we have to do a little bit of cleaning before we do some analysis. Let's look at the current values:

In [ ]:
df["Home Team"].value_counts()

We'll build a dictionary holding our "official" names, and for each a list of regular expressions that should hopefully catch all of the above:

In [ ]:
teams = {
    "A-Team": [".*Advis.*", ".*A.*Team.*"],
    "Central": [".*Central.*",".*Senior.*"],
    "Health": [".*Health.*",".*HEP.*"],
    "International": [".*International.*"],
    "Impact Incubator": [".*Impact.*",".*I I.*"],
    "Leaving Well": [".*Leaving.*"],
    "DigiLabs": [".*Digi.*Labs.*"],
}
     
df["CleanedTeam"] = df["Home Team"]
for team, patterns in teams.items():
    for pattern in patterns:
        df["CleanedTeam"] = df["CleanedTeam"].str.replace(pattern, team, flags=re.IGNORECASE)
                
df["CleanedTeam"].value_counts()

# Let's see where people are from

In [ ]:

def make_autopct(values):
    def my_autopct(pct):
        total = sum(values)
        val = int(round(pct*total/100.0))
        return '{p:.0f}%  ({v:d})'.format(p=pct,v=val)
    return my_autopct


counts = df["CleanedTeam"].value_counts()
counts.plot("pie", label="", autopct=make_autopct(counts))

# How many days do people typically spend in the office 

In [ ]:
counts = df["In a typical week, how many days are you in the office?"].value_counts(sort=False,bins=range(0,6))
counts.plot("pie", label="", autopct=make_autopct(counts),counterclock=False,labels=range(1,6))

In [ ]:
def stacked_bar(df, col1, col2="CleanedTeam", name1=None, name2=None, pct=True, show_table=False):
    counts = df.copy().reset_index()
#     counts[col1] = df[col1].round()
    counts = counts[[col2,col1,"ID"]].groupby([col1,col2]).count().unstack()
    counts = counts.fillna(0).astype(int)

    columns = list(counts.columns.levels[1])
    counts.columns = columns
    counts.columns.name = name2
    counts.index.name = name1

    # Convert to percent
    for c in counts.columns:
        total = counts[c].sum()
        counts[c] = (counts[c]/total)*100

    if show_table:
        display(counts)
    
    
    counts.T.plot(kind='bar', stacked=True)
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., title=name1)
    
stacked_bar(df, col1="In a typical week, how many days are you in the office?", name1="Days", show_table=True)

# How much time do people actually spend at their desk

In [ ]:
counts = df["At your desk"].value_counts()
counts.plot("pie", label="", autopct=make_autopct(counts))

In [ ]:
# Rename series so that they sort nicely
counts = df.copy()
counts["At your desk"] = counts["At your desk"].replace({"Once a day": "1-Once", "Several times a day": "2-Several","Most of the day": "3-Most"})

stacked_bar(counts, col1="At your desk", show_table=True)

# Not many people love hotdesking...

In [ ]:
counts = df["How do you feel about hotdesking / sharing a desk?"].value_counts()
counts.plot("pie", label="", autopct=make_autopct(counts))

In [ ]:
# Rename series so that they sort nicely
counts = df.copy()
counts["How do you feel about hotdesking / sharing a desk?"] = counts["How do you feel about hotdesking / sharing a desk?"] \
    .replace({"Hate it": "1-Hate", 
             "Don't care": "2-IDC",
             "Love it": "3-\N{Heavy Black Heart}"})

stacked_bar(counts, col1="How do you feel about hotdesking / sharing a desk?", show_table=True)